In [1]:
import cv2
import os
from os import listdir
import numpy as np
from stitching import Stitcher

In [2]:
images = [] # list of images that make up the panorama
folder_dir = "lounge" # folder name that contains the captured images
for image in sorted(os.listdir(folder_dir)):
    if (image.endswith(".jpg")): # grabbing all the jpg files in the target folder
        img = cv2.imread(folder_dir+"/"+image)
        images.append(img)

# setting up the OpenStitching Stitcher class
stitcher = Stitcher(
    detector="sift", # using the SIFT detector instead of ORB
    confidence_threshold=0.01,  
    matcher_type='homography',
    range_width=-1,
    crop=True # cropping the panorama to remove extra waved edges
)

try:
    # Add error handling for the stitching process
    panorama = stitcher.stitch(images)
    if panorama is not None:
        cv2.imwrite(folder_dir+'_pano.jpg', panorama)
    else:
        print("Stitching failed - no panorama was created")
except Exception as e:
    print(f"Error during stitching: {str(e)}")

In [3]:
def resize(path, new_path):
    image = cv2.imread(path) # read in the unpadded panorama
    height, width, dim = image.shape 
    bot_mid = np.mean(image[-1:, :, :], axis=(0, 1)) # average color of bottom row
    top_mid = np.mean(image[:1, :, :], axis=(0, 1)) # average color of top row
    new_height = width//2 # equirectangular image needs to be 2:1
    new_image = bot_mid*np.ones((new_height, width, dim), dtype=np.uint8) # creating blank image
    new_image[:new_height//2, :, :] = top_mid*np.ones((new_height//2, width, dim), dtype=np.uint8)
    bot, top = (new_height-height)//2, (new_height+height)//2 # center original image in center
    new_image[bot:top, : , :] = image # inserting original image
    cv2.imwrite(new_path, new_image) # saving the new image

resize(folder_dir+"_pano.jpg", folder_dir+"_resized.jpg")